In [6]:
# imports

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import plot_confusion_matrix, recall_score,\
    accuracy_score, precision_score, f1_score

from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImPipeline

In [61]:
# importing 'testing' data
test_vals = pd.read_csv('../data/test_vals.csv')

In [62]:
# exploring 'testing' data
test_vals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14850 entries, 0 to 14849
Data columns (total 40 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     14850 non-null  int64  
 1   amount_tsh             14850 non-null  float64
 2   date_recorded          14850 non-null  object 
 3   funder                 13981 non-null  object 
 4   gps_height             14850 non-null  int64  
 5   installer              13973 non-null  object 
 6   longitude              14850 non-null  float64
 7   latitude               14850 non-null  float64
 8   wpt_name               14850 non-null  object 
 9   num_private            14850 non-null  int64  
 10  basin                  14850 non-null  object 
 11  subvillage             14751 non-null  object 
 12  region                 14850 non-null  object 
 13  region_code            14850 non-null  int64  
 14  district_code          14850 non-null  int64  
 15  lg

In [12]:
# importing training data
training_labels = pd.read_csv('../data/training_labels.csv')

In [13]:
# exploring training target data
training_labels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            59400 non-null  int64 
 1   status_group  59400 non-null  object
dtypes: int64(1), object(1)
memory usage: 928.2+ KB


In [14]:
# importing training features data
training_vals = pd.read_csv('../data/training_vals.csv')

In [15]:
#exploring training features data
training_vals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 40 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     59400 non-null  int64  
 1   amount_tsh             59400 non-null  float64
 2   date_recorded          59400 non-null  object 
 3   funder                 55765 non-null  object 
 4   gps_height             59400 non-null  int64  
 5   installer              55745 non-null  object 
 6   longitude              59400 non-null  float64
 7   latitude               59400 non-null  float64
 8   wpt_name               59400 non-null  object 
 9   num_private            59400 non-null  int64  
 10  basin                  59400 non-null  object 
 11  subvillage             59029 non-null  object 
 12  region                 59400 non-null  object 
 13  region_code            59400 non-null  int64  
 14  district_code          59400 non-null  int64  
 15  lg

In [16]:
# merging target and features of training data into one dataframe
data = pd.merge(training_vals, training_labels)

In [17]:
# exploring new dataframe
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59400 entries, 0 to 59399
Data columns (total 41 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     59400 non-null  int64  
 1   amount_tsh             59400 non-null  float64
 2   date_recorded          59400 non-null  object 
 3   funder                 55765 non-null  object 
 4   gps_height             59400 non-null  int64  
 5   installer              55745 non-null  object 
 6   longitude              59400 non-null  float64
 7   latitude               59400 non-null  float64
 8   wpt_name               59400 non-null  object 
 9   num_private            59400 non-null  int64  
 10  basin                  59400 non-null  object 
 11  subvillage             59029 non-null  object 
 12  region                 59400 non-null  object 
 13  region_code            59400 non-null  int64  
 14  district_code          59400 non-null  int64  
 15  lg

In [18]:
# exploring nulls
data.isna().sum()

id                           0
amount_tsh                   0
date_recorded                0
funder                    3635
gps_height                   0
installer                 3655
longitude                    0
latitude                     0
wpt_name                     0
num_private                  0
basin                        0
subvillage                 371
region                       0
region_code                  0
district_code                0
lga                          0
ward                         0
population                   0
public_meeting            3334
recorded_by                  0
scheme_management         3877
scheme_name              28166
permit                    3056
construction_year            0
extraction_type              0
extraction_type_group        0
extraction_type_class        0
management                   0
management_group             0
payment                      0
payment_type                 0
water_quality                0
quality_

In [19]:
# exploring unique values 'scheme_name'
data['scheme_name'].value_counts()

K                                               682
None                                            644
Borehole                                        546
Chalinze wate                                   405
M                                               400
                                               ... 
Nyakayenzi gravity/point source water supply      1
Old mkuu water supply                             1
Ka                                                1
Mkindu R                                          1
Malolo dam                                        1
Name: scheme_name, Length: 2696, dtype: int64

In [20]:
# exploring number of unique values in 'scheme_name'
len(data['scheme_name'].unique())

2697

In [21]:
# exploring unique values 'scheme_management'
data['scheme_management'].value_counts()

VWC                 36793
WUG                  5206
Water authority      3153
WUA                  2883
Water Board          2748
Parastatal           1680
Private operator     1063
Company              1061
Other                 766
SWC                    97
Trust                  72
None                    1
Name: scheme_management, dtype: int64

In [22]:
# exploring columns with 10 or less unique values for potentially OHE
col_less_10 = []
for column in data.columns:
    if len(data[column].unique()) <= 10:
        col_less_10.append(column)

print(col_less_10)
len(col_less_10)

['basin', 'public_meeting', 'recorded_by', 'permit', 'extraction_type_class', 'management_group', 'payment', 'payment_type', 'water_quality', 'quality_group', 'quantity', 'quantity_group', 'source', 'source_type', 'source_class', 'waterpoint_type', 'waterpoint_type_group', 'status_group']


18

### Exploring unique values in each column

In [23]:
data['district_code'].value_counts()

1     12203
2     11173
3      9998
4      8999
5      4356
6      4074
7      3343
8      1043
30      995
33      874
53      745
43      505
13      391
23      293
63      195
62      109
60       63
0        23
80       12
67        6
Name: district_code, dtype: int64

In [25]:
data['region_code'].value_counts()

11    5300
17    5011
12    4639
3     4379
5     4040
18    3324
19    3047
2     3024
16    2816
10    2640
4     2513
1     2201
13    2093
14    1979
20    1969
15    1808
6     1609
21    1583
80    1238
60    1025
90     917
7      805
99     423
9      390
24     326
8      300
40       1
Name: region_code, dtype: int64

In [26]:
data['lga'].value_counts()

Njombe          2503
Arusha Rural    1252
Moshi Rural     1251
Bariadi         1177
Rungwe          1106
                ... 
Moshi Urban       79
Kigoma Urban      71
Arusha Urban      63
Lindi Urban       21
Nyamagana          1
Name: lga, Length: 125, dtype: int64

In [ ]:
data['ward'].value_counts()

In [27]:
data['permit'].value_counts()

True     38852
False    17492
Name: permit, dtype: int64

In [28]:
data['construction_year'].value_counts()

0       20709
2010     2645
2008     2613
2009     2533
2000     2091
2007     1587
2006     1471
2003     1286
2011     1256
2004     1123
2012     1084
2002     1075
1978     1037
1995     1014
2005     1011
1999      979
1998      966
1990      954
1985      945
1980      811
1996      811
1984      779
1982      744
1994      738
1972      708
1974      676
1997      644
1992      640
1993      608
2001      540
1988      521
1983      488
1975      437
1986      434
1976      414
1970      411
1991      324
1989      316
1987      302
1981      238
1977      202
1979      192
1973      184
2013      176
1971      145
1960      102
1967       88
1963       85
1968       77
1969       59
1964       40
1962       30
1961       21
1965       19
1966       17
Name: construction_year, dtype: int64

In [66]:
data['extraction_type'].value_counts()

gravity                      26780
nira/tanira                   8154
other                         6430
submersible                   4764
swn 80                        3670
mono                          2865
india mark ii                 2400
afridev                       1770
ksb                           1415
other - rope pump              451
other - swn 81                 229
windmill                       117
india mark iii                  98
cemo                            90
other - play pump               85
walimi                          48
climax                          32
other - mkulima/shinyanga        2
Name: extraction_type, dtype: int64

In [68]:
data['extraction_type_group'].value_counts()

gravity            26780
nira/tanira         8154
other               6430
submersible         6179
swn 80              3670
mono                2865
india mark ii       2400
afridev             1770
rope pump            451
other handpump       364
other motorpump      122
wind-powered         117
india mark iii        98
Name: extraction_type_group, dtype: int64

In [69]:
data['extraction_type_class'].value_counts()

gravity         26780
handpump        16456
other            6430
submersible      6179
motorpump        2987
rope pump         451
wind-powered      117
Name: extraction_type_class, dtype: int64

In [71]:
data['management'].value_counts()

vwc                 40507
wug                  6515
water board          2933
wua                  2535
private operator     1971
parastatal           1768
water authority       904
other                 844
company               685
unknown               561
other - school         99
trust                  78
Name: management, dtype: int64

In [72]:
data['management_group'].value_counts()

user-group    52490
commercial     3638
parastatal     1768
other           943
unknown         561
Name: management_group, dtype: int64

In [73]:
data['payment'].value_counts()

never pay                25348
pay per bucket            8985
pay monthly               8300
unknown                   8157
pay when scheme fails     3914
pay annually              3642
other                     1054
Name: payment, dtype: int64

In [74]:
data['payment_type'].value_counts()

never pay     25348
per bucket     8985
monthly        8300
unknown        8157
on failure     3914
annually       3642
other          1054
Name: payment_type, dtype: int64

In [75]:
data['water_quality'].value_counts()

soft                  50818
salty                  4856
unknown                1876
milky                   804
coloured                490
salty abandoned         339
fluoride                200
fluoride abandoned       17
Name: water_quality, dtype: int64

In [76]:
data['quality_group'].value_counts()

good        50818
salty        5195
unknown      1876
milky         804
colored       490
fluoride      217
Name: quality_group, dtype: int64

In [77]:
data['quantity'].value_counts()

enough          33186
insufficient    15129
dry              6246
seasonal         4050
unknown           789
Name: quantity, dtype: int64

In [78]:
data['quantity_group'].value_counts()

enough          33186
insufficient    15129
dry              6246
seasonal         4050
unknown           789
Name: quantity_group, dtype: int64

In [79]:
data['source'].value_counts()

spring                  17021
shallow well            16824
machine dbh             11075
river                    9612
rainwater harvesting     2295
hand dtw                  874
lake                      765
dam                       656
other                     212
unknown                    66
Name: source, dtype: int64

In [80]:
data['source_type'].value_counts()

spring                  17021
shallow well            16824
borehole                11949
river/lake              10377
rainwater harvesting     2295
dam                       656
other                     278
Name: source_type, dtype: int64

In [81]:
data['source_class'].value_counts()

groundwater    45794
surface        13328
unknown          278
Name: source_class, dtype: int64

In [82]:
data['waterpoint_type'].value_counts()

communal standpipe             28522
hand pump                      17488
other                           6380
communal standpipe multiple     6103
improved spring                  784
cattle trough                    116
dam                                7
Name: waterpoint_type, dtype: int64

In [84]:
data['waterpoint_type_group'].value_counts()

communal standpipe    34625
hand pump             17488
other                  6380
improved spring         784
cattle trough           116
dam                       7
Name: waterpoint_type_group, dtype: int64

### Making a list of all the columns that are either redundant or not useful to model

In [30]:
drop_cols_list = ['id','date_recorded', 'latitude', 'longitude', 'wpt_name', 'num_private', 'region_code', 'district_code', 'lga', 'ward', 'subvillage', 'public_meeting', 'recorded_by', 'extraction_type', 'extraction_type_group', 'management_group', 'payment_type', 'water_quality', 'quantity_group', 'source_type', 'source_class', 'waterpoint_type']

In [31]:
len(drop_cols_list)

22

In [95]:
# creating dataframe without columns mentioned above
df1 = data.drop(drop_cols_list, axis=1)

In [96]:
df1.head()

,amount_tsh,funder,gps_height,installer,basin,region,population,scheme_management,scheme_name,permit,construction_year,extraction_type_class,management,payment,quality_group,quantity,source,waterpoint_type_group,status_group
0,6000.0,Roman,1390,Roman,Lake Nyasa,Iringa,109,VWC,Roman,False,1999,gravity,vwc,pay annually,good,enough,spring,communal standpipe,functional
1,0.0,Grumeti,1399,GRUMETI,Lake Victoria,Mara,280,Other,NaN,True,2010,gravity,wug,never pay,good,insufficient,rainwater harvesting,communal standpipe,functional
2,25.0,Lottery Club,686,World vision,Pangani,Manyara,250,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,vwc,pay per bucket,good,enough,dam,communal standpipe,functional
3,0.0,Unicef,263,UNICEF,Ruvuma / Southern Coast,Mtwara,58,VWC,NaN,True,1986,submersible,vwc,never pay,good,dry,machine dbh,communal standpipe,non functional
4,0.0,Action In A,0,Artisan,Lake Victoria,Kagera,0,NaN,NaN,True,0,gravity,other,never pay,good,seasonal,rainwater harvesting,communal standpipe,functional


In [97]:
df1.shape

(59400, 19)

### Dealing with nulls and missing values and creating different iterations of the dataset based on this

In [99]:
# creating dataframe 'df2' with no nulls
df2 = df1.dropna()
df2.shape

(28986, 19)

In [100]:
# creating dataframe 'df3' which removes all rows with '0' in 'construction_year' column
df3 = df2.drop(df2.loc[df2['construction_year']==0].index)

In [101]:
df3.shape

(22710, 19)

In [63]:
# renaming dataset
data_all = df3

### Creating dataset without 'scheme_name' column to allow for all data (minus nulls and '0's)

In [102]:
# creating dataframe 'df4' which drops column 'scheme_name'
df4 = df1.drop('scheme_name', axis=1)
df4.shape

(59400, 18)

In [104]:
# creating dataframe 'df5' without 'scheme_name' and dropping nulls
df5 = df4.dropna()
df5.shape

(51281, 18)

In [105]:
# creating dataframe 'data6' without 'scheme_name' column and removed all rows with '0' in 'contruction_year' column
df6 = df5.drop(df5.loc[df5['construction_year']==0].index)
df6.shape

(34298, 18)

In [106]:
# renaming dataframe
data_no_sname = df6

### Creating dataset without 'contruction_year' column to allow for all data (minus nulls)

In [107]:
# creating dataframe 'df4' which drops column 'construction year'
df7 = df1.drop('construction_year', axis=1)
df7.shape

(59400, 18)

In [108]:
# creating dataframe 'data5' without 'construction_year' and dropping nulls
df8 = df7.dropna()
df8.shape

(28986, 18)

In [110]:
# renaming dataframe
data_no_constr = df8

### Creating dataset without 'scheme_name' or 'construction_year' columns (minus nulls)

In [111]:
df9 = df1.drop(['scheme_name', 'construction_year'], axis=1)
df9.shape

(59400, 17)

In [112]:
df10 = df9.dropna()
df10.shape

(51281, 17)

In [113]:
# renaming dataframe
data_no_sname_no_constr = df10

### Creating training and testing data for each dataframe ('all columns', 'no_scheme_name', 'no_contruction', no_scheme_name_or_construction')

In [118]:
df_list = {'data_all': data_all, 'data_no_sname': data_no_sname, 'data_no_constr': data_no_constr, 'data_no_sname_no_constr': data_no_sname_no_constr}
df_dic = {}

for key, value in df_list.items():
    X = value.drop('status_group', axis=1)
    y = value['status_group']
    z = key
    df_dic["X_train_" + z], df_dic["X_test_" + z], df_dic["y_train_" + z], df_dic["y_test_" + z] = train_test_split(X, y)

In [122]:
df_dic.keys()

dict_keys(['X_train_data_all', 'X_test_data_all', 'y_train_data_all', 'y_test_data_all', 'X_train_data_no_sname', 'X_test_data_no_sname', 'y_train_data_no_sname', 'y_test_data_no_sname', 'X_train_data_no_constr', 'X_test_data_no_constr', 'y_train_data_no_constr', 'y_test_data_no_constr', 'X_train_data_no_sname_no_constr', 'X_test_data_no_sname_no_constr', 'y_train_data_no_sname_no_constr', 'y_test_data_no_sname_no_constr'])

### Creating subpipelines for models

In [181]:
def grab_numeric(df):
    df = pd.DataFrame(df)
    return df.select_dtypes(include=['float', 'int'])

In [182]:
GrabNum = FunctionTransformer(grab_numeric)

In [183]:
subpipe_num = Pipeline(steps=[('grab_num', GrabNum), ('scaler', StandardScaler())])

In [184]:
def grab_categorical(df):
    df = pd.DataFrame(df)
    return df.select_dtypes(include=['object'])

In [185]:
GrabCat = FunctionTransformer(grab_categorical)

In [186]:
subpipe_cat = Pipeline(steps=[('grab_cat', GrabCat), ('ohe', OneHotEncoder(sparse=False, handle_unknown='ignore'))])

In [187]:
col_trans = ColumnTransformer(transformers=[('subpipe_num', subpipe_num, X.columns), ('subpipe_cat', subpipe_cat, X.columns)])

### Logistic regression model

In [188]:
logreg_pipe = Pipeline(steps=[('col_trans', col_trans), ('logreg', LogisticRegression())])

In [189]:
logreg_pipe.fit(df_dic['X_train_data_all'], df_dic['y_train_data_all'])

/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('col_trans',
                 ColumnTransformer(transformers=[('subpipe_num',
                                                  Pipeline(steps=[('grab_num',
                                                                   FunctionTransformer(func=<function grab_numeric at 0x7fcbfdd29280>)),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['amount_tsh', 'funder', 'gps_height', 'installer', 'basin', 'region',
       'population', 'scheme_management', 'permit', 'extraction_type_class',
       'management',...
                                                                   FunctionTransformer(func=<function grab_categorical at 0x7fcbfda371f0>)),
                                                                  ('ohe',
                                                                   OneHotEncoder(

pandas.core.series.Series